This is more or less a test to figure out how to create repositories, etc

In [2]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Desktop/fall 2023/cee 4750/final project/Elliot-Leah-Mark-final-project`


In [3]:
Pkg.add("Plots")
using Plots # plotting
Pkg.add("LinearAlgebra")
using LinearAlgebra

   Resolving package versions...


  No Changes to `~/Desktop/fall 2023/cee 4750/final project/Elliot-Leah-Mark-final-project/Project.toml`
  No Changes to `~/Desktop/fall 2023/cee 4750/final project/Elliot-Leah-Mark-final-project/Manifest.toml`


Below starts the actual code. Steps as defined in project update.

## 0. cell matrices ##

In [7]:
length_cell=300; #m, FIXME
height_watershed_cells=10; #number of cells, FIXME
width_watershed_cells=13; #number of cells, FIXME

number_cells_total=height_watershed_cells*width_watershed_cells; #number of cells
area_cell= length_cell^2; #m^2

map_area=[]; #area coefficient of each cell, FIXME
map_ag=[]; #ag coefficient of each cell, FIXME
#curve numbers, from soil series -> hydraulic soil types -> weighted avg approximation based on crop type
#perhaps ADJUST, FIXME
map_CN=[];
map_distance=[]; #distance to outlet, FEET, maybe automate, FIXME
map_elevation=[]; #height above outlet, FEET, FIXME

## 1. calculating manure and phosphorus ##

In [5]:
depth_applied=.5; #inches, ADJUST
depth_mixed=.5; #inches, ADJUST

d_interacting= 0.5*depth_applied/depth_mixed*2.54/100; #m

density_manure=63*16.0185; #kg/m^3

#from Pagliari, 2014
phosphorus_in_manure_amount=10.55 #g/kg
inorg_fract=.695;

## 2. calculating idealized storm ##

In [6]:
precip_storm_total=25.4; #mm, to be tweaked
duration_storm=60; #min, to be tweaked

intensity_storm=precip_storm_total/duration_storm; #mm/min

## 3. defining calculations for each cell ##

In [ ]:
t_runoff_start_matrix=zeros(height_watershed, width_watershed); #minutes
t_c_matrix=zeros(height_watershed, width_watershed); #minutes
vol_matrix=zeros(height_watershed, width_watershed); #m^3
C_matrix=zeros(height_watershed, width_watershed); #g/m^3

rate_vol_outlet=1.566 *49.3/38.6*ones(1000); #m^3/sec, array per minute
C_outlet=11.396*ones(1000); #mg/L, array per minute

for i in height_watershed
    for j in width_watershed
        area_cell_inside=area_cell*map_area[i,j];
        area_ag=area_cell*map_ag[i,j];
        CN=map_cn[i,j];
        distance=map_distance[i,j];
        
        S=25400/CN-254; #mm
        Ia=0.2*s; #mm

        depth_Q=(precip_storm_total-Ia)^2/(precip_storm_total-Ia+S); #mm
        vol_Q=depth_Q*area_cell_inside/1000; #m^3
        vol_Q_ag=vol_Q*area_ag/area_cell_inside;
        vol_Q_ag_liters=vol_Q_ag*1000;

        t_runoff_start=duration_storm*Ia/depth_Q; #min
        rate_vol_Q=vol_Q/(duration_storm-t_runoff_start)/60; #m^3/sec

        M_interacting_manure=d_interacting*density_manure*area_ag; #kg
        M_runoff_ag=vol_Q*1000; #kg

        M_interacting_inorg=M_interacting_manure*.733; #g
        
        M_dissolved_inorg=0; #g

        #piecewise function built from plot in Pagliari, 2014
        if 0<vol_Q_ag_liters/M_interacting_manure<=12.5
            M_dissolved_inorg=((vol_Q_ag_liters/M_interacting_manure-1.7)/10.8*.44+.07)/100*M_interacting_inorg; #g
        else if 12.5<vol_Q_ag_liters/M_interacting_manure <= 50.8
            M_dissolved_inorg=((vol_Q_ag_liters/M_interacting_manure-12.5)/(50.8-12.5)*.46+.51)/100*M_interacting_inorg; #g
        else if 50.8<vol_Q_ag_liters/M_interacting_manure
            M_dissolved_inorg=((vol_Q_ag_liters/M_interacting_manure-50.8)/(101.7/50.8)*(0-.06)+.97)/100*M_interacting_inorg; #g
        end

        t_runoff_start_matrix[i,j]=t_runoff_start;
        t_c_matrix[i,j]=0.0078*map_distance[i,j]^0.77/(map_elevation[i,j]/map_distance[i,j])^0.385; #minutes
        vol_matrix[i,j]=vol_Q;
        C_matrix[i,j]=M_dissolved_inorg/vol_Q; #g/m^3=mg/L
        
        for k=1:length(rate_vol_outlet)
            if t_runoff_start+t_c_matrix[i,j]<i<t_runoff_start+duration_storm+t_c_matrix[i,j]
                rate_vol_outlet[k] += rate_vol_Q;
                C_outlet[k] += C_matrix[i,j]*rate_vol/rate_vol_outlet[k];
            end
        end
        end
    end
end

mass_total_ag=dot(vol_matrix, C_matrix);